# 第三章：提示词工程 (Prompt Engineering)

> 💡 **核心观点**：即使是相同模型，不同的提示词会产生不同的结果

## 1. 提示词工程概述

### 1.1 定义

**提示词工程（Prompt Engineering）**：通过设计和优化输入语言提示，引导 LLM 或智能体系统按照预期目标生成内容、执行操作的技术和方法。

---

## 2. 提示工程核心概念

### 2.1 提示词的组成

提示词通常由以下两部分组成：

- **系统提示（System Prompt）**
  - 影响每一次对话
  - 设定模型的角色、行为准则和输出格式

- **用户提示（User Prompt）**
  - 影响本次对话
  - 包含具体的任务指令和上下文信息

### 2.2 内容组织原则

- **内容划分 & 结构划分**
  - 关键内容放在**开头和结尾**
  - 利用首因效应和近因效应提高模型关注度

### 2.3 基本原则

设计提示词时应遵循以下基本原则：

1. ✅ **清晰**：表达明确，避免歧义
2. ✅ **具体**：提供详细的指令和要求
3. ✅ **上下文适配**：考虑背景信息和场景
4. ✅ **可验证性**：输出结果可被验证和评估
5. ✅ **适度冗余**：关键信息可适当重复强调

---

## 3. 提示工程常见技术

### 3.1 零样本学习（Zero-Shot Learning）

**角色扮演（Role Playing）**

- **原理**：在提示词中明确指定模型扮演的身份角色或职业
- **效果**：影响模型的语言风格、思考角度和行为模式
- **示例**：
  ```
  你是一位经验丰富的软件架构师，请分析以下代码...
  ```

### 3.2 少样本学习（Few-Shot Learning）

**上下文学习（In-Context Learning）**

- **原理**：在提示词中放入示例（examples）
- **进阶**：可以与思维链（CoT）结合，放入包含推理逻辑的例子
- **优势**：让模型通过示例学习任务模式和期望输出格式

### 3.3 思维链（Chain of Thought, CoT）

- **核心思想**：`Let's think step by step`
- **方法**：引导模型逐步推理，展示思考过程
- **效果**：提高复杂推理任务的准确性和可解释性

### 3.4 自我一致（Self-Consistency）

- **原理**：多次生成答案，选择出现次数最多的答案
- **适用场景**：需要高准确性的任务
- **方法**：通过多次采样，利用"多数投票"机制提高可靠性

### 3.5 自我反思（Self-Reflection）

**相关技术**：

- `self-reflection`
- `ReACT` (Reasoning + Acting)
- `Actor-Evaluator`
- `Self-Reflexion`

**原理**：让模型对自己的输出进行评估和反思，迭代改进

---

## 4. 提示工程调优方法及实践

### 4.1 提示工程调优流程（HumanRF）

#### 核心步骤

1. **拆解任务，指令化表达**
   - 将复杂任务分解为简单步骤
   - 用清晰的指令描述每个步骤

#### 迭代优化流程

```
初版提示词
    ↓
输出结果
    ↓
优化步骤：
  • 分步提示化
  • 加入示例
  • 观察输出偏差
  • 添加约束条件
  • 调整措辞
  • 角色设定优化
    ↓
下一版提示词
```

### 4.2 优化要点

| 优化方向 | 具体方法 |
|---------|---------|
| **结构化** | 分步提示化，明确任务步骤 |
| **示例化** | 加入少样本示例，引导输出格式 |
| **约束化** | 添加约束条件，限制输出范围 |
| **角色化** | 设定合适的角色，影响输出风格 |
| **措辞优化** | 调整表达方式，提高指令清晰度 |

---

## 5. 总结

提示词工程是充分发挥大语言模型能力的关键技术。通过合理设计提示词结构、运用各种技术方法，并持续迭代优化，可以显著提升模型在特定任务上的表现。

**关键要点**：
- 🎯 清晰、具体、可验证的提示词设计
- 🔄 结合零样本、少样本、思维链等技术
- 📈 通过迭代优化不断提升效果
    


    



---

## 6. 实践代码

### 6.1 环境准备

在使用提示工程之前，首先需要准备 API 调用环境。以下是一个通用的 API 调用函数：

In [ ]:
import json
import os
from pathlib import Path
from openai import OpenAI

def load_config(config_path=None):
    """
    从JSON文件加载API配置
    自动查找配置文件：优先查找 HandsOn/config.json
    
    Args:
        config_path: 配置文件路径，如果为None则自动查找
    
    Returns:
        dict: 包含所有API配置的字典，如果失败返回None
    """
    # 如果未指定路径，自动查找配置文件
    if config_path is None:
        # 可能的配置文件位置（按优先级排序）
        possible_paths = [
            "config.json",  # 当前目录（如果notebook在HandsOn目录运行）
            "HandsOn/config.json",  # HandsOn子目录
            "../HandsOn/config.json",  # 上级目录的HandsOn文件夹
            os.path.join(os.getcwd(), "HandsOn", "config.json"),  # 绝对路径
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                config_path = path
                break
        
        if config_path is None:
            print("❌ 错误: 找不到配置文件")
            print("   请确保在 HandsOn/ 目录下存在 config.json 文件")
            print("   可以从 HandsOn/config.json.example 复制并填写你的API密钥")
            return None
    
    try:
        # 转换为绝对路径并读取
        config_path = os.path.abspath(config_path)
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)
        print(f"✅ 成功加载配置文件: {config_path}")
        return config
    except FileNotFoundError:
        print(f"❌ 错误: 找不到配置文件 {config_path}")
        print("   请复制 HandsOn/config.json.example 为 HandsOn/config.json 并填写你的API密钥")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ 错误: JSON格式错误 - {e}")
        print(f"   请检查 {config_path} 文件的JSON语法")
        return None

# 加载配置（自动查找 HandsOn/config.json）
config = load_config()

def get_response(system_prompt, user_prompt, platform="openai", model=None, temperature=0.0, top_p=1.0, max_tokens=2048):
    """
    调用大语言模型 API 获取回复
    
    参数:
        system_prompt: 系统提示词
        user_prompt: 用户提示词
        platform: 平台标识，用于从config读取配置（可选: "openai", "chatglm", "deepseek", "qwen"）
        model: 模型名称（可选，如果不提供则使用配置文件中的默认模型）
        temperature: 温度参数
        top_p: Top-p 采样参数
        max_tokens: 最大生成 token 数
    
    返回:
        模型的回复内容
    """
    # 从配置文件读取API配置
    if config is None:
        return "❌ 错误: 配置文件未加载，请检查 HandsOn/config.json 文件"
    
    platform_config = config.get(platform, {})
    api_key = platform_config.get("api_key")
    base_url = platform_config.get("base_url")
    default_model = platform_config.get("model")
    
    # 使用提供的model或配置文件中的默认model
    model = model or default_model
    
    if not api_key or not base_url or not model:
        return f"❌ 错误: 未找到 {platform} 的完整API配置。请检查 HandsOn/config.json 文件"
    
    try:
        client = OpenAI(api_key=api_key, base_url=base_url)  # 实例化通信客户端
        response = client.chat.completions.create(
            model=model,  # 基座模型
            messages=[  # 对话内容
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
            stream=False
        )
        return response.choices[0].message.content  # 返回智能体的回答
    except Exception as e:
        return f"❌ 错误: API调用失败 - {str(e)}"

---

### 6.2 系统提示与用户提示实践

让我们通过实际示例来理解系统提示和用户提示的区别和作用：

In [ ]:
# 示例 1: 基础系统提示和用户提示
system_prompt = "你是一名助教，需要温柔的回答同学们的问题。"  # 系统提示
user_prompt = "在做提示工程时，什么内容适合放入系统提示、什么内容适合放入用户提示？" 

response = get_response(system_prompt, user_prompt)
print(response)

可以看到，模型以温柔的语气回答了问题。现在让我们在系统提示中加入格式要求，看看模型如何按照指定格式输出：

In [ ]:
# 示例 2: 带格式要求的系统提示
system_prompt = """
你是一名助教，需要温柔的回答同学们的问题。
你回答问题时必须先对问题进行充分的解释，再通过举例支持和形象化你的解释，最后对问题做出一句话的总结。
你的回答必须遵循以下格式：
【解释】：
【示例】：
【总结】：
"""  
user_prompt = "在做提示工程时，什么内容适合放入系统提示、什么内容适合放入用户提示？" 

response = get_response(system_prompt, user_prompt)
print(response)

可以看到，在系统提示中加入规定格式后，模型可以按照设定的格式回答问题，我们可以方便地通过字符串匹配获得解释或者示例。

> 💡 **提示**：在进行单轮问答时，将内容放在系统提示和用户提示并没有太大的区别，但是在多轮问答时，将不变的规则放入系统提示，变化的规则放入用户提示中会方便许多。

---

### 6.3 清晰指令原则实践

让我们通过对比示例来理解清晰指令的重要性：

In [ ]:
system_prompt = "你是一个营养师，擅长为人们制定健康饮食计划。"

# 含糊指令
user_prompt_blur = "怎样吃才能减肥？"

# 清晰指令
user_prompt_clear = "我想减肥，目标是半年内从120斤减到100斤，请为我制定一个包含早餐、午餐和晚餐的健康饮食计划，确保每天摄入的热量不超过1500卡路里。"

answer_blur = get_response(system_prompt, user_prompt_blur)
print("=" * 60)
print("含糊指令回答：")
print("=" * 60)
print(answer_blur)

print("\n" + "=" * 60)
print("清晰指令回答：")
print("=" * 60)
answer_clear = get_response(system_prompt, user_prompt_clear)
print(answer_clear)

可以看到，当使用含糊指令时，模型回答内容松散，缺乏针对性；而当我们使用清晰的指令阐述需求时，模型便可以根据用户需要做出相应的回答。

---

### 6.4 角色扮演实践

角色扮演是提示工程中非常重要且实用的技巧。让我们通过对比示例来理解角色扮演的效果：

In [ ]:
user_prompt = "请简要回答如何治疗感冒"

# 无角色扮演
direct_answer = get_response("", user_prompt)
print("=" * 60)
print("无角色扮演：")
print("=" * 60)
print(direct_answer)

# 有角色扮演
system_prompt = "你是一位经验丰富的医生，擅长治疗各种常见疾病，尤其是感冒。你会非常专业、耐心地回答患者的问题，有效安抚患者情绪，并提供详细的治疗建议。"
response = get_response(system_prompt, user_prompt)
print("\n" + "=" * 60)
print("有角色扮演：")
print("=" * 60)
print(response)

总结而言，通过在提示中设定模型角色，能够增强回复专业性，模型会调动该角色相关的内容和表达习惯，输出更专业的内容。

---

### 6.5 思维链（Chain of Thought）实践

思维链技术通过引导模型进行深度思考后再回答，有效提高回复的准确性和逻辑性。让我们通过一个概率问题来对比使用思维链前后的效果：

In [ ]:
question = "有甲、乙、丙三个盒子，甲盒中有一个白球和两个黑球，乙盒中有两个白球和一个黑球，丙盒中有三个白球和三个黑球，掷一枚骰子决定盒子。若出现的点数为1，2，3，选择甲盒，若出现的点数为4，选择乙盒，若出现的点数为5，6，选择丙盒。再从选中的盒子中任意选取一球。求：当取出的球为白球时，此球来自甲盒的概率？"

# 无思维链：直接给出答案
system_prompt = "直接给出问题的答案，禁止任何分析和思考。"
response = get_response(system_prompt, question)
print("=" * 60)
print("无思维链（直接回答）：")
print("=" * 60)
print(response)

# 有思维链：逐步思考
system_prompt = "逐步思考并给出问题答案"
response = get_response(system_prompt, question)
print("\n" + "=" * 60)
print("有思维链（逐步思考）：")
print("=" * 60)
print(response)

使用思维链后，模型给出了详细的推理过程，并得出了正确的答案。思维链技术对于复杂推理任务非常有效。

---

### 6.6 上下文学习（In-Context Learning）实践

上下文学习通过在输入中添加适当的示例文本来提高模型能力。以下是一个情感分类的示例：

In [ ]:
prompt = """
将文本分类为正面、中性、或负面情感。请根据以下示例进行分类：
文本：我今天过得很开心！
情感：正面
文本：这部电影真是太糟糕了。
情感：负面
文本：我人为这个餐厅的食物还可以。
情感：中性

对以下文本进行分类：
文本：这个地方风景优美，鸟语花香，气候宜人。
情感：
"""
response = get_response("", prompt)
print(response)

通过提供示例，模型能够学习任务模式并按照期望的格式输出结果。

---

### 6.7 提示工程调优完整案例：电影推荐系统

下面我们通过一个完整的电影推荐系统案例，演示如何通过迭代优化提示词来提升模型输出质量。

#### 6.7.1 初版提示词

In [ ]:
MOVIE_RECOMMEND_SYSTEM_PROMPT_CHINESE = "你是一个专业的电影推荐系统，你的任务是根据用户的观影历史，从候选电影列表中推荐最符合用户口味的电影。"

item_lists = """
1. 电影ID: 001, 名称: 《蒙娜丽莎的微笑》
2. 电影ID: 002, 名称: 《肖申克的救赎》
3. 电影ID: 003, 名称: 《阿甘正传》
4. 电影ID: 004, 名称: 《扬名立万》
5. 电影ID: 005, 名称: 《这个杀手不太冷静》
6. 电影ID: 006, 名称: 《美丽人生》
7. 电影ID: 007, 名称: 《电锯惊魂》
8. 电影ID: 008, 名称: 《移动迷宫》
"""

candidates = """
1. 电影ID: 007, 名称: 《盗梦空间》
2. 电影ID: 008, 名称: 《泰坦尼克号》
3. 电影ID: 009, 名称: 《抓娃娃》
4. 电影ID: 010, 名称: 《哪吒之魔童降世》
5. 电影ID: 011, 名称: 《疯狂的石头》
6. 电影ID: 012, 名称: 《神探夏洛克》
7. 电影ID: 013, 名称: 《疯狂动物城》
8. 电影ID: 014, 名称: 《穿普拉达的女王》
"""

MOVIE_RECOMMEND_PROMPT_CHINESE = """
**输入信息**
1. 用户最近的观影记录，按时间排序，越靠后表示越近期观看，每一项为电影ID和电影名。
{item_list}
2. 待推荐的候选电影列表，每一项为电影ID和电影名。
{candidates}

**回答格式**：你必须严格按照以下要求格式回答

解释: [你的思考和分析过程]
推荐: [名称1, 名称2, 名称3, ... 按照推荐程度降序排列电影的名称]
"""

response = get_response(
    MOVIE_RECOMMEND_SYSTEM_PROMPT_CHINESE,
    MOVIE_RECOMMEND_PROMPT_CHINESE.format(
        item_list=item_lists,
        candidates=candidates
    ))    
print(response)

在该回答中，模型根据用户看过的电影分析了用户的喜好，但是没有考虑到时间因素。我们针对这一问题改进提示词，进一步明确完成任务时需要关注的内容。

#### 6.7.2 改进版提示词（考虑时间因素）

In [ ]:
MOVIE_RECOMMEND_PROMPT_CHINESE = """
**输入信息**
1. 用户最近的观影记录，按时间排序，越靠后表示越近期观看，每一项为电影ID和电影名。
{item_list}
2. 待推荐的候选电影列表，每一项为电影ID和电影名。
{candidates}

**任务**
你需要根据用户最近的观影记录，通过分析每部电影的风格和内容，以及观看该电影的时间顺序，整合分析用户可能的电影喜好以及近期偏好变化，推荐出用户可能最想观看的下一部电影。输出时，请排序待推荐的候选电影列表，要求按照推荐的优先级排序，推荐程度越高位置越靠前。输出列表中仅包含电影的名称。

**回答格式**：你必须严格按照以下要求格式回答

解释: [你的思考和分析过程]
推荐: [名称1, 名称2, 名称3, ... 按照推荐程度降序排列电影的名称]
"""

response = get_response(
    MOVIE_RECOMMEND_SYSTEM_PROMPT_CHINESE,
    MOVIE_RECOMMEND_PROMPT_CHINESE.format(
        item_list=item_lists,
        candidates=candidates
    ))    
print(response)

这一次，模型不仅分析了用户的电影偏好，并且分析出了用户的偏好变化，考虑的信息更加全面，推理结果也更加合理。但是输出格式却出现了问题。这在实际做提示工程时是很容易出现的，因为大模型底层逻辑中的注意力机制，在输入文本过长时，可能会产生忽视关键信息的问题。

#### 6.7.3 最终版提示词（格式优化）

In [ ]:
MOVIE_RECOMMEND_PROMPT_CHINESE = """
**输入信息**
1. 用户最近的观影记录，按时间排序，越靠后表示越近期观看，每一项为电影ID和电影名。
{item_list}
2. 待推荐的候选电影列表，每一项为电影ID和电影名。
{candidates}

**任务**
你需要根据用户最近的观影记录，通过分析每部电影的风格和内容，以及观看该电影的时间顺序，整合分析用户可能的电影喜好以及近期偏好变化，推荐出用户可能最想观看的下一部电影。输出时，请排序待推荐的候选电影列表，要求按照推荐的优先级排序，推荐程度越高位置越靠前。输出列表中仅包含电影的名称。

**回答格式**：你必须严格按照以下要求格式回答，否则你的答案会被视为无效。

解释: [详细描述你的思考和分析过程]
推荐: [名称1, 名称2, 名称3, ... 按照推荐程度降序排列电影的名称]
"""

response = get_response(
    MOVIE_RECOMMEND_SYSTEM_PROMPT_CHINESE,
    MOVIE_RECOMMEND_PROMPT_CHINESE.format(
        item_list=item_lists,
        candidates=candidates
    ))    
print(response)

通过强调格式要求的重要性（"否则你的答案会被视为无效"），模型现在能够更好地遵循输出格式。

> 💡 **总结**：提示调优过程为：初版提示 → 观察模型表现，找出问题点 → 针对性改写提示，逐轮试验调整 → 直至输出符合预期。提示工程是一个实验性很强、非常依赖试错的过程，没有"万能好用"的初版提示，调优迭代是提示工程中最核心、最真实的工作方式。

---

## 7. 代码要点总结

### 7.1 提示词设计要点
- ✅ **系统提示**：设定角色、行为准则和输出格式
- ✅ **用户提示**：包含具体的任务指令和上下文信息
- ✅ **清晰具体**：避免模糊表达，提供详细指令
- ✅ **格式约束**：明确输出格式要求，便于后续处理

### 7.2 常用技术实践
- ✅ **角色扮演**：通过设定角色提升专业性
- ✅ **思维链**：引导模型逐步推理，提高准确性
- ✅ **上下文学习**：通过示例引导模型学习任务模式

### 7.3 调优流程
- ✅ **观察输出偏差**：分析模型输出与预期目标的差距
- ✅ **迭代优化**：通过反复试验不断调整提示词
- ✅ **格式强化**：在关键位置强调格式要求

---

## 8. 注意事项

1. **API 配置**：确保 HandsOn/config.json 文件已正确配置，包含所需的 API 密钥和平台信息
2. **提示词长度**：注意提示词长度限制，避免超出模型上下文窗口
3. **格式要求**：在提示词中明确格式要求，并在关键位置强调
4. **迭代优化**：提示工程需要反复试验和优化，要有耐心
5. **成本控制**：注意 API 调用成本，合理设置 max_tokens 等参数